# DATA PREPARATION

## SETUP

In [45]:
import pandas as pd
pd.set_option('display.max_columns', None)

## LOAD DATA

In [ ]:
from ucimlrepo import fetch_ucirepo 

'''# metadata 
print(diabetes_130_us_hospitals_for_years_1999_2008.metadata) 
  
# variable information 
print(diabetes_130_us_hospitals_for_years_1999_2008.variables) '''
  
# fetch dataset 
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296) 
  
# data (as pandas dataframes) 
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets 

# create complete real_data
diabetes = pd.DataFrame(X)
diabetes["readmitted"] = y

# visualize data
diabetes.head()

## EXPLORE & PREPROCESS DATA

### Dimensions

In [ ]:
# dimensions
print(f"Dimension: {diabetes.shape}")

###  Check columns values & distribution

In [ ]:
# SINGLE COLUMN: check columns values & distribution
def visualize_columns_distributions(df):
    for col in df.columns:
        print(f"\n\nColumn: {col}")

        # Combine proportions into a DataFrame for easy comparison
        data =  pd.DataFrame({
            'Values': df[col].value_counts(dropna=False),
            'Proportions': df[col].value_counts(normalize = True, dropna=False)
        }).fillna(0)

        # print data
        print(data)
    
# call to columns_distibution function
visualize_columns_distributions(diabetes)    

Data is not balanced they are more caucasian (%74) than any other race.

###  Generalize 'Nan' race values to 'Other'

In [ ]:
# Generalize Nan race as 'Other'
import numpy as np
diabetes.loc[diabetes["race"].isin([np.nan ,'Other']), "race"] = "Other"

# validate change
diabetes.race.value_counts() 

Hispanic and Asian can be generalized as other race but this way still be considerably imbalanced. 

###  Check 'Unknown/Invalid' gender values

In [ ]:
# 'Unknown/Invalid' data
diabetes[diabetes['gender'] == 'Unknown/Invalid'] 

####  Correct 'Unknown/Invalid' gender values

Only 3 registry, not possible to define gender, best option would be to remove them.

In [ ]:
# removing  'Unknown/Invalid' gender data
print(f"Shape before drop: {diabetes.shape}")
diabetes = diabetes.drop(diabetes[diabetes["gender"] == 'Unknown/Invalid'].index)

# validating results (only 3 less)
print(f"Shape after drop: {diabetes.shape}")

###  Check for 'missing values'

In [ ]:
# nulls per columns (percentage)
diabetes.isna().sum() * 100 / len(diabetes)

Weight column has many null values (96.858387 %), best option would be to remove.

####  Drop "weight" column

In [ ]:
# remove weight column 
print(f"Columns before remove {len(diabetes.columns)}")
diabetes = diabetes.drop('weight', axis=1)
print(f"Columns after remove {len(diabetes.columns)}")

The columns `payer_code` (39.56%) and `medical_specialty` (49.08%) have many null values. Let's take actions on them.    

#### Check missing value nature

Are those missing values related? How they are related?

In [ ]:
# Filter data for payer_code null
df1 = diabetes[diabetes["payer_code"].isnull()]

print(f"Filtering data for `payer_code` null values new null relation is: \n {df1.isna().sum() * 100 / len(df1)}") 

Effectively great part of the missing values are related. Once `payer_code` is missing in (40%) of the cases `medical_specialty` is also missing.

Check if the removing of this rows is feasible.

In [ ]:
print(f"Data without payer code represent: {round(df1.shape[0] * 100 / diabetes.shape[0],2)} % of total data")
print(f"If they are removed means reduce dimension from: {diabetes.shape[0]} lines to {diabetes.shape[0] - df1.shape[0]}.")

There is still enough data for synthetic data creation, so we will delete them. However, for analysis purposes, removing 40 thousand lines is not optimal at all.

#### Correct missing values

In [ ]:
# avoid payer_code null data
df1 = diabetes[~diabetes["payer_code"].isnull()]

# reasign again to diabetes
diabetes = df1

# validate results => no null values
print(f"Payer_code null values: {diabetes['payer_code'].isna().sum()}")

Check again null values.

In [ ]:
# nulls per columns (percentage)
diabetes.isna().sum() * 100 / len(diabetes)

Remove those columns that exceed 50% of the null values.

In [ ]:
# stablish missing proportion
missing_proportion = diabetes.isna().sum() * 100 / len(diabetes)

# get columns that meet this proportion
columns_to_drop = missing_proportion[missing_proportion > 50].index

# remove columns that exceed these proportion
print(f"Columns: {columns_to_drop} exceed 50% of missing proportion")
print(f"Actual dataframe shape: {diabetes.shape}.\nRemoving columns that exceed the proportion.")
new_df = diabetes.drop(columns=columns_to_drop)
diabetes = new_df
print(f"Dimension after removing data: {diabetes.shape}.")

###  Check for 'variability'

Columns that have only one value

In [ ]:
def columns_without_variability(df):    
    """
    Function that is responsible to determine which columnns has no variability (those which has only 1 value).    
    Parameters:
        df (pd.DataFrame): The original DataFrame.

    Returns:
        list(): list of variables without variability.
    """    
    sobran = []

    cols = df.columns
    for col in cols:
        if len(df[col].unique()) < 2:
            print(f"Column: `{col}` unique values: {df[col].unique()}")
            sobran.append(col)

    return sobran

# obtain column list without variability
cols_without_variability = columns_without_variability(diabetes)

# print result
print(f"Invariant columns: {cols_without_variability}")

#### Remove columns without variability.

In [ ]:
# remove columns
print(f"Columns without variability: {cols_without_variability}")
print(f"Columns before remove {len(diabetes.columns)}")
diabetes = diabetes.drop(columns = cols_without_variability)
print(f"Columns after remove {len(diabetes.columns)}")

### Detect 'sensitive columns'

In [ ]:
# display all columns
print(f"\columns: {diabetes.columns}\n")

# identify identity sensible data: 
sensitive_columns = ['race', 'gender', 'age', 'admission_type_id','discharge_disposition_id','admission_source_id','payer_code', 'medical_specialty']
print(f"\nSensitive columns: {sensitive_columns}\n")

### Detect 'single value entries'

Columns that present a singularization risk, columns that have more than one value but only a single instance of one value.

In [ ]:
def determine_single_value_entries(df):
    rows_to_check = []  # List to hold the rows matching the criteria
    cols_to_check = []
    for col in df.columns:
        # Get the value counts for the column
        value_counts = df[col].value_counts()

        # Check if exactly one value has a count of 1
        if (value_counts == 1).sum() == 1:
            # Get the value that appears exactly once
            single_value = value_counts[value_counts == 1].index[0]
            # add column name
            cols_to_check.append(col)
            
            # Select rows where this single value appears
            matching_rows = df[df[col] == single_value]
            
            # Append these rows to the list
            rows_to_check.append(matching_rows)

    # Concatenate all the matching rows into a single dataframe (if needed)
    result_df = pd.concat(rows_to_check, ignore_index=True) if rows_to_check else pd.DataFrame()

    return result_df,cols_to_check

# determine single value entries
matching_rows_df, cols_to_check = determine_single_value_entries(diabetes)

# check data relevancy
for col in cols_to_check:
    value_counts = diabetes[col].value_counts(dropna = False)
    print(f"Single value entry in column {col} :  {value_counts[value_counts == 1].index[0]}")
    #print(f"Distribution \n{value_counts}")


With synthetic data anonymization, this singularization risk can be diminished, but since there are few records, they can be analyzed to determine if they can be removed.

#### Check 'single value entries'

The columns `metformin-pioglitazone` and `glimepiride-pioglitazone` show a `steady` value. Do they correspond to the same individual?

In [ ]:
# Filter from matching_rows_df those only they have 1 Steady instances
steadys = ["metformin-pioglitazone","glimepiride-pioglitazone"] 
val ="Steady"

# Filter rows where exactly one of the specified columns has the value 'Steady'
filtered_df = diabetes.loc[
    (diabetes[steadys[0]] == val).astype(int) +
    (diabetes[steadys[1]] == val).astype(int) +
    (diabetes[steadys[2]] == val).astype(int) == 1
]

# show results
print(filtered_df)

Check sensitive_columns for any reidentification risk.

In [ ]:
print(filtered_df[sensitive_columns])

Check also this columns values variability:

In [ ]:
for col in steadys:
    print(f"\nColumn: {col} \nunique values: {diabetes[col].value_counts()}")

These two steady cases clearly pose a reidentification risk due to their singularization. If they are removed, the `metformin-pioglitazone`and `glimepiride-pioglitazone`columns will lose their variability and should also be removed.

Check the other `single value entries` to evaluate if they should be removed. 

Check 'admission_source_id' = 13 case:

In [ ]:
# risk of reindetifiction better to be removed.
diabetes[diabetes['admission_source_id'] == 13] 

Check 'payer_code' : 'FR'

In [ ]:
# risk of reindetifiction better to be removed.
diabetes[diabetes['payer_code'] == 'FR'] 

Evaluate columns shape to determine if `steady` columns need to be removed or not.

In [ ]:
diabetes.shape

#### Take action on 'single value entries'

Remove steady rows and columns.

In [ ]:
# Remove rows and columns as they do not have variability after removing
print(f"Actual dimension: {diabetes.shape}")
print(f"Removing rows: {filtered_df.index}")
diabetes = diabetes.drop(filtered_df.index)
print(f"After dimension: {diabetes.shape}")
print("Checking column variability")
cols_without_variability = columns_without_variability(diabetes)
print(f"Removing columns: {cols_without_variability}")
diabetes = diabetes.drop(columns = cols_without_variability)
print(f"Later dimension: {diabetes.shape}")

Remove the other single value registries.

In [ ]:
# Remove single 2 rows
print(f"Current dimensions: {diabetes.shape}")

# Calculate how many rows will be removed
rows_to_remove = len(diabetes[(diabetes['admission_source_id'] == 13) ^ 
                              (diabetes['payer_code'] == 'FR')])
print(f"Removing {rows_to_remove} rows.")

# Drop the rows based on conditions
diabetes = diabetes.drop(diabetes[(diabetes['admission_source_id'] == 13) ^ 
                                  (diabetes['payer_code'] == 'FR')].index)

print(f"Updated dimensions: {diabetes.shape}")

###  Check dtype uniformity: 

In [ ]:
# data information
print(f"\nData information: {diabetes.dtypes}\n")

#### Check numerical data

In [ ]:
# get updated list
num_cols = diabetes.select_dtypes(include='int64')

# Check column values, correspond to dtypes
for cat in num_cols.columns:
    print(f"\nColumn: {cat} values: {diabetes[cat].unique()}")

#### Change numerical columns to categoricals

Columns `admission_type_id`, `discharge_disposition_id` and `admission_source_id`are not numerical columns. They are categorical columns because their values represent distinct types of admissions or sources.

In [ ]:
# Change columns type
cols_to_change = ["admission_type_id","discharge_disposition_id", "admission_source_id"]
diabetes[cols_to_change] =  diabetes[cols_to_change].astype(str)
diabetes.info()

#### Check categorical data 

In [ ]:
# optimize memory use changing object to string
categorical_cols = diabetes.select_dtypes('object').columns.tolist()

# Check column values, correspond to dtypes
for cat in categorical_cols:
    print(f"\nColumn: {cat} values: {diabetes[cat].unique()}")

#### Check categorical column pair relations

In [ ]:
import itertools

# get categorical column pairs
categorical_col_pair = list(itertools.combinations(categorical_cols, 2))       

# visualize data relations
for pair in categorical_col_pair:
    print(f"\n{pair[0]} distribution per {pair[1]}")
    print(f"{diabetes.groupby(pair[0])[pair[1]].value_counts(dropna= False).unstack().fillna(0)}")

### Save preprocessed file

In [64]:
import os

# create folder
tmp_folder = "./tmp_folder"
os.makedirs(tmp_folder, exist_ok=True)

# save data
diabetes.to_parquet(os.path.join(tmp_folder,"preprocessed_file.parquet"),engine="pyarrow",index=False)